In [2]:
!pip install opencv-python
!pip install dlib
!pip install face_recognition 
!pip install tensorflow
!pip install keras
!pip install spotipy

In [3]:
import requests
from flask import Flask, jsonify, request, redirect
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import cv2
import time
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import threading


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

cid = 'e9376ef50e0d40dba61b28c2d296a516'
secret = '822f153a2a6849bc98ca722031d17761'
redirect_uri = 'http://localhost:8888/callback'
cache_path = '.spotify_cache'

# Spotify 인증을 수행하고 토큰을 캐시에 저장
auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, cache_path=cache_path)
sp = spotipy.Spotify(auth_manager=auth_manager)

# 인증을 수행하여 토큰을 얻음
token_info = auth_manager.get_access_token(as_dict=True)
print(f"Token information: {token_info}")

# 토큰 정보를 파일에 저장
with open('spotify_token.txt', 'w') as f:
    f.write(token_info['access_token'])




/var/folders/v8/gp3klmn91vx_hphdwtxgwng80000gn/T/ipykernel_28018/2286567012.py:14: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = auth_manager.get_access_token(as_dict=True)


Token information: {'access_token': 'BQBxoLUCoG2flIHfSnhPQgacbNJO8BhdzAIvweTiKpOSUweMGZK1oRKi7H7xuFWXhBOstXRtmTl0AM-0O7yGbJz9LcBfAGCjr8AdC86aCdYnlpmV95m1Xjlss9X9tQtZEWoMu0KpliYc2FzTMEgkenYBrwlc9MKqsVDb3Y8YOLnFy7bNbn2C5GlRejVzxVJ946gqvko1VPvLrWXL2oOCLHPfCg', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': None, 'expires_at': 1721103014, 'refresh_token': 'AQCZKh0GxS-7fFqWVy13MzNXMDs8_GSHOC0mjFD-pRVsJqPfUxSU5KtP3Vy6oFxz1iHrPIPRfnF1qLnOQQpo0yzfq3zj7m2FOCtbdrt5HY6aegs92yKD-DEI2FwZjdXOic4'}


In [2]:

# 스포티파이 API 인증 정보
cid = 'e9376ef50e0d40dba61b28c2d296a516'
secret = '822f153a2a6849bc98ca722031d17761'
redirect_uri = 'http://localhost:8888'
cache_path = '.spotify_cache'

# 파일에서 저장된 토큰을 읽어옴
with open('spotify_token.txt', 'r') as f:
    access_token = f.read().strip()

# Spotify 클라이언트를 초기화
sp = spotipy.Spotify(auth=access_token)

# 감정 모델 경로 설정
file_path = '/Users/gimseongmin/Library/Mobile Documents/com~apple~CloudDocs/C&E/연구실/ai경진대회_sw개발/무제 폴더/emotion_model.h5'
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 파일 경로 확인 및 모델 로드
if os.path.exists(file_path):
    print("File exists.")
    emotion_model = load_model(file_path)
else:
    print("File does not exist.")
    emotion_model = None

def get_song_recommendation(emotion, sp):
    emotion_to_features = {
        'Happy': {'danceability': 0.8, 'energy': 0.8, 'valence': 0.9},
        'Sad': {'danceability': 0.2, 'energy': 0.2, 'valence': 0.1},
        'Angry': {'danceability': 0.7, 'energy': 0.9, 'valence': 0.2},
        'Relaxed': {'danceability': 0.5, 'energy': 0.3, 'valence': 0.8},
        'Disgust': {'danceability': 0.1, 'energy': 0.1, 'valence': 0.1},
        'Fear': {'danceability': 0.3, 'energy': 0.3, 'valence': 0.3},
        'Surprise': {'danceability': 0.6, 'energy': 0.6, 'valence': 0.6},
        'Neutral': {'danceability': 0.5, 'energy': 0.5, 'valence': 0.5}
    }
    features = emotion_to_features.get(emotion, {'danceability': 0.5, 'energy': 0.5, 'valence': 0.5})
    recommendations = sp.recommendations(seed_genres=['pop'], limit=5, 
                                         target_danceability=features['danceability'],
                                         target_energy=features['energy'],
                                         target_valence=features['valence'])
    if recommendations['tracks']:
        tracks = recommendations['tracks']
        return [f"{track['name']} by {track['artists'][0]['name']}" for track in tracks]
    else:
        return ["No recommendations available"]

def handle_emotion_recommendation(emotion, sp):
    try:
        songs = get_song_recommendation(emotion, sp)
        print(f"Recommended songs for emotion '{emotion}': {songs}")
    except Exception as e:
        print(f"Error in thread: {e}")

if emotion_model is not None:
    # 웹캠 설정
    cap = cv2.VideoCapture(0)

    # 감정 지속 시간 체크 변수 초기화
    last_emotion = None
    emotion_start_time = time.time()

    try:
        while True:
            # 웹캠 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break

            # 얼굴 인식
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            face_locations = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

            current_emotion = None

            for (x, y, w, h) in face_locations:
                # 얼굴 부분 자르기
                face = gray_frame[y:y+h, x:x+w]
                face = cv2.resize(face, (48, 48))
                face = face.astype('float32') / 255.0
                face = img_to_array(face)
                face = np.expand_dims(face, axis=0)

                # 감정 예측
                emotion_prediction = emotion_model.predict(face)[0]
                predicted_emotion_index = emotion_prediction.argmax()
                if predicted_emotion_index < len(emotion_labels):
                    current_emotion = emotion_labels[predicted_emotion_index]
                else:
                    current_emotion = None

                # 감정 지속 시간 체크
                if current_emotion in emotion_labels:
                    if current_emotion == last_emotion:
                        if time.time() - emotion_start_time >= 5:
                            # 5초간 같은 감정일 때 노래 추천
                            threading.Thread(target=handle_emotion_recommendation, args=(current_emotion, sp)).start()
                            emotion_start_time = time.time()  # 타이머 리셋
                    else:
                        last_emotion = current_emotion
                        emotion_start_time = time.time()  # 타이머 리셋

                # 결과 화면에 표시
                if current_emotion:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, current_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # 화면에 프레임 출력
            cv2.imshow('Emotion and Song Recommendation', frame)

            # 'q' 키를 누르면 종료
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # 웹캠 릴리스 및 창 닫기
        cap.release()
        cv2.destroyAllWindows()
else:
    print("Emotion model is not loaded. Exiting.")


NameError: name 'spotipy' is not defined